In [1]:
import os
import warnings
import pandas as pd
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

warnings.filterwarnings("ignore")
encoder = SentenceTransformer("all-MiniLM-L6-v2")
load_dotenv()

client = QdrantClient(os.getenv("QDRANT_HOST"), api_key=os.getenv("QDRANT_API_KEY"))

# Loading and cleaning data
This [dataset](https://www.kaggle.com/datasets/zynicide/wine-reviews) contains approximately 130k reviews from the Wine Enthusiast

Once cleaned we will have around 120k.

In [2]:
df = pd.read_csv("winemag-data-130k-v2.csv")

In [3]:
wines = df.copy()
wines = wines.drop(
    [
        "Unnamed: 0",
        "designation",
        "province",
        "region_1",
        "region_2",
        "taster_name",
        "taster_twitter_handle",
        "winery",
    ],
    axis=1,
)
wines = wines.dropna(subset=["country", "price", "variety"])

In [4]:
wines.head()

,country,description,points,price,title,variety
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir
5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot


In [5]:
wines.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120915 entries, 1 to 129970
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      120915 non-null  object 
 1   description  120915 non-null  object 
 2   points       120915 non-null  int64  
 3   price        120915 non-null  float64
 4   title        120915 non-null  object 
 5   variety      120915 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.5+ MB


# Create a collection

In [ ]:
client.create_collection(
    collection_name="wine_reviews",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,
    ),
)

In [ ]:
# Document class to structure data
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata


# Convert DataFrame rows into Document objects
def df_to_documents(df):
    documents = []
    for _, row in df.iterrows():
        metadata = {
            "country": row["country"],
            "points": row["points"],
            "price": row["price"],
            "title": row["title"],
            "variety": row["variety"],
        }
        document = Document(page_content=row["description"], metadata=metadata)
        documents.append(document)
    return documents


docs = df_to_documents(wines)

In [ ]:
points = [
    models.PointStruct(
        id=idx,
        vector=encoder.encode(doc.page_content).tolist(),
        payload={"metadata": doc.metadata, "page_content": doc.page_content},
    )
    for idx, doc in enumerate(docs)
]

In [ ]:
client.upload_points(
    collection_name="wine_reviews",
    points=points,
)

# Test search

In [6]:
hits = client.search(
    collection_name="wine_reviews",
    query_vector=encoder.encode("Quinta dos Avidagos 2011").tolist(),
    limit=3,
)

for hit in hits:
    print(hit.payload["metadata"]["title"], "score:", hit.score)

Aveleda 2010 Follies Quinta da Agueira Touriga Nacional (Beiras) score: 0.46982175
Quinta da Romaneira 2013 Sino da Romaneira Red (Douro) score: 0.43031913
Quinta da Romaneira 2013 Sino da Romaneira Red (Douro) score: 0.43031913


# Test filtering

In [7]:
# query filter
hits = client.search(
    collection_name="wine_reviews",
    query_vector=encoder.encode("Night Sky").tolist(),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.country", match=models.MatchValue(value="US")
            ),
            models.FieldCondition(
                key="metadata.price", range=models.Range(gte=15.0, lte=30.0)
            ),
            models.FieldCondition(
                key="metadata.points", range=models.Range(gte=90, lte=100)
            ),
        ]
    ),
    limit=3,
)

for hit in hits:
    print(
        hit.payload["metadata"]["title"],
        "\nprice:",
        hit.payload["metadata"]["price"],
        "\npoints:",
        hit.payload["metadata"]["points"],
        "\n\n",
    )

Ballentine 2010 Fig Tree Vineyard Petite Sirah (St. Helena) 
price: 28.0 
points: 91 


Seven Angels 2012 St. Peter of Alcantara Vineyard Zinfandel (Paso Robles) 
price: 29.0 
points: 92 


Jamieson Canyon 1999 Cabernet Sauvignon (Napa Valley) 
price: 20.0 
points: 91 




# Self-querying with LangChain

In [8]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_qdrant import Qdrant

handler = ConsoleCallbackHandler()
llm = ChatOpenAI(temperature=0, model="gpt-4o")
# llm = OpenAI(temperature=0)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Qdrant(client, collection_name="wine_reviews", embeddings=embeddings)

In [9]:
metadata_field_info = [
    AttributeInfo(
        name="country",
        description="The country that the wine is from",
        type="string",
    ),
    AttributeInfo(
        name="points",
        description="The number of points WineEnthusiast rated the wine on a scale of 1-100",
        type="integer",
    ),
    AttributeInfo(
        name="price",
        description="The cost for a bottle of the wine",
        type="float",
    ),
    AttributeInfo(
        name="variety",
        description="The grapes used to make the wine",
        type="string",
    ),
]

document_content_description = "Brief description of the wine"

retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info
)

In [11]:
response = retriever.invoke(
    "Which US wines are priced between 15 and 30 and have points above 90?"
)
response

[Document(page_content='An outstanding value, the latest release of this wine dazzles with bold, black cherry and chocolate mocha flavors. The focus and definition throughout are exceptional also. This is a gem at a more than fair tariff.', metadata={'country': 'US', 'points': 91, 'price': 28.0, 'title': 'Dobbes Family Estate 2014 Grand Assemblage Pinot Noir (Willamette Valley)', 'variety': 'Pinot Noir', '_id': 10604, '_collection_name': 'wine_reviews'}),
 Document(page_content='This is an amazingly fresh and fruity tank-fermented wine, imparting a subtle hint of grass before unleashing sublime layers of melon and apricot alongside measured, zesty acidity. New winemaker Chris Kajani is taking things in a refreshing, aim-for-the-top direction with this bottling.', metadata={'country': 'US', 'points': 92, 'price': 30.0, 'title': "Bouchaine 2013 Chêne d'Argent Estate Vineyard Chardonnay (Carneros)", 'variety': 'Chardonnay', '_id': 102251, '_collection_name': 'wine_reviews'}),
 Document(pa

In [12]:
for resp in response:
    print(
        resp.metadata["title"],
        "\n price:",
        resp.metadata["price"],
        "points:",
        resp.metadata["points"],
        "\n\n",
    )

Dobbes Family Estate 2014 Grand Assemblage Pinot Noir (Willamette Valley) 
 price: 28.0 points: 91 


Bouchaine 2013 Chêne d'Argent Estate Vineyard Chardonnay (Carneros) 
 price: 30.0 points: 92 


Conn Creek 2014 Chardonnay (Carneros) 
 price: 25.0 points: 92 


Longoria 2011 Pinot Grigio (Santa Barbara County) 
 price: 19.0 points: 91 




# Tracing to see filters in action

In [13]:
retriever.invoke(
    "Which US wines are priced between 15 and 30 and have points above 90?",
    {"callbacks": [handler]},
)

[chain/start] [retriever:Retriever > chain:query_constructor] Entering Chain run with input:
{
  "query": "Which US wines are priced between 15 and 30 and have points above 90?"
}
[chain/start] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "Which US wines are priced between 15 and 30 and have points above 90?"
}
[chain/end] [retriever:Retriever > chain:query_constructor > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:Retriever > chain:query_constructor > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to compare to document contents\n    \"filter\": string \\ logic

[Document(page_content='An outstanding value, the latest release of this wine dazzles with bold, black cherry and chocolate mocha flavors. The focus and definition throughout are exceptional also. This is a gem at a more than fair tariff.', metadata={'country': 'US', 'points': 91, 'price': 28.0, 'title': 'Dobbes Family Estate 2014 Grand Assemblage Pinot Noir (Willamette Valley)', 'variety': 'Pinot Noir', '_id': 10604, '_collection_name': 'wine_reviews'}),
 Document(page_content='This is an amazingly fresh and fruity tank-fermented wine, imparting a subtle hint of grass before unleashing sublime layers of melon and apricot alongside measured, zesty acidity. New winemaker Chris Kajani is taking things in a refreshing, aim-for-the-top direction with this bottling.', metadata={'country': 'US', 'points': 92, 'price': 30.0, 'title': "Bouchaine 2013 Chêne d'Argent Estate Vineyard Chardonnay (Carneros)", 'variety': 'Chardonnay', '_id': 102251, '_collection_name': 'wine_reviews'}),
 Document(pa